In [26]:
import numpy as np
import cv2 as cv
import pandas as pd
import os

In [27]:
pathCsvOptic = '/Users/nathansadoun/Desktop/INSA/ML/Video_classification/MediaEval_ClassificationTask/dataSetOptic'

In [28]:
def analyseOptic(video_path, pathCsv):
    
    cap = cv.VideoCapture(video_path)

    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                        qualityLevel = 0.3,
                        minDistance = 7,
                        blockSize = 7 )

    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15, 15),
                    maxLevel = 2,
                    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

    # Create some random colors
    color = np.random.randint(0, 255, (100, 3))

    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    listeFlow = []

    while(1):
        ret, frame = cap.read()
        if not ret:
            print('No frames grabbed!')
            break

        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        listeFlow.append(p1)

        # Select good points
        if p1 is not None:
            good_new = p1[st==1]
            good_old = p0[st==1]

        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    df = pd.DataFrame(listeFlow)
    # header = ["box_conf", "box_x", "box_y", "box_w", "box_h", "box_x_n", "box_y_n", "box_w_n", "box_h_n",
    #             "1_x", "1_y", "1_conf", "1_x_n", "1_y_n",
    #             "2_x", "2_y", "2_conf", "2_x_n", "2_y_n",
    #             "3_x", "3_y", "3_conf", "3_x_n", "3_y_n",
    #             "4_x", "4_y", "4_conf", "4_x_n", "4_y_n",
    #             "5_x", "5_y", "5_conf", "5_x_n", "5_y_n",
    #             "6_x", "6_y", "6_conf", "6_x_n", "6_y_n",
    #             "7_x", "7_y", "7_conf", "7_x_n", "7_y_n",
    #             "8_x", "8_y", "8_conf", "8_x_n", "8_y_n",
    #             "9_x", "9_y", "9_conf", "9x_n", "9_y_n",
    #             "10_x", "10_y", "10_conf", "10_x_n", "10_y_n",
    #             "11_x", "11_y", "11_conf", "11_x_n", "11_y_n",
    #             "12_x", "12_y", "12_conf", "12_x_n", "12_y_n",
    #             "13_x", "13_y", "13_conf", "13_x_n", "13_y_n",
    #             "14_x", "14_y", "14_conf", "14_x_n", "14_y_n",
    #             "15_x", "15_y", "15_conf", "15_x_n", "15_y_n",
    #             "16_x", "16_y", "16_conf", "16_x_n", "16_y_n",
    #             "17_x", "17_y", "17_conf", "17_x_n", "17_y_n",
    #         ]
    # df = df.set_axis(header,axis=1) # adding headers to the dataframe
    df.to_csv(pathCsv, sep=';', decimal='.', index=False) # store keypoint csv

    # Release the video capture object and close the display window
    cap.release()

In [29]:
def analyseOptic_dataset(pathToData: str, pathToOutput: str):
    listeErreur = []

    for folder in sorted(os.listdir(pathToData)):
        if (not folder.startswith(".")) and os.path.isdir(os.path.join(pathToData, folder)):
            if folder != "test":
                pathToCSVFolder = os.path.join(pathToOutput, folder) # create a CSV folder for train and validation datasets
                if not os.path.exists(pathToCSVFolder):
                    os.mkdir(pathToCSVFolder)
                pathToFolder = os.path.join(pathToData, folder)
                for subFolder in sorted(os.listdir(pathToFolder)): 
                    if not subFolder.startswith(".") and os.path.isdir(os.path.join(pathToFolder, subFolder)):
                        pathToCSVSubFolder = os.path.join(pathToCSVFolder, subFolder) # create a CSV folder for each class of strokes
                        if not os.path.exists(pathToCSVSubFolder):
                            os.mkdir(pathToCSVSubFolder)
                        pathToSubFolder = os.path.join(pathToFolder, subFolder)
                        for file in sorted(os.listdir(pathToSubFolder)):
                            if (not file.startswith(".")) and (file.endswith(".mp4")):
                                pathToFile = os.path.join(pathToSubFolder, file)
                                CSVFileName = file[:len(file) - 4]
                                pathToCSV = os.path.join(pathToCSVSubFolder, CSVFileName+".csv")
                                if not os.path.exists(pathToCSV):
                                    print(pathToFile)
                                    try:
                                        analyseOptic(pathToFile, pathToCSV)
                                    except:
                                        print("Erreur fichier: "+pathToFile)
                                        listeErreur.append(pathToFile)
    return listeErreur

In [30]:
analyseOptic('/Users/nathansadoun/Desktop/INSA/ML/Video_classification/MediaEval_ClassificationTask/videoTest.mp4', '/Users/nathansadoun/Desktop/INSA/ML/Video_classification/MediaEval_ClassificationTask/videoTest.csv')

No frames grabbed!


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (218,) + inhomogeneous part.

: 